<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularization

*Author: Alexander Del Toro Barba*

## Overview

* Regularization is a technique for preventing over-fitting by penalizing a model for having large weights
* Weight regularization was borrowed from penalized regression models in statistics



![Regularization Types](https://raw.githubusercontent.com/deltorobarba/repo/master/5907C4B3-6EC5-40CD-BB51-A5AB75C3DC71.jpeg)

Source: ['Getting started with Regression'](https://medium.com/@savannahar68/getting-started-with-regression-a39aca03b75f)

## L1 Regularization

* Sum of the absolute weights. Gives sparse solutions, since it does not take all features
* **Synonyms**: Lasso, Manhatten distance, least absolute deviations (LAD), least absolute errors (LAE)
* **Advantages**: less influenced by outliers (robust). Can shrink some coefficients to zero while lambda increases, performing variable selection. generates sparse feature vectors (Sparse: only very few entries in a matrix or vector is non-zero. L1-norm has property of producing many coefficients with zero values or very small values with few large coefficients). Sparse is sometimes good eg. in high dimensional classification problems. sparsity properties: calculation more computationally efficient.
* **Disadvantages**: L1 regularization doesn’t easily work with all forms of training. gives a solution with more large residuals, and a lot of zeros in the solution.



## L2 Regularization

* Sum of the squared weights. Is the most common type of regularization, also called simply “weight decay,” with values often on a logarithmic scale between 0 and 0.1, such as 0.1, 0.001, 0.0001, etc.
* **Synonyms**: Weight Decay, Ridge Regression, KQ-Methode, kleinste Quadrate, Tikhonov regularization, Euclidean distance, least squares error (LSE)
* **Advantages**: Shrinks all the coefficient by the same proportions, but eliminates none. Leads to small distributed weights in neural networks. The L2 regularization heavily penalizes "peaky" weight vectors and prefers diffuse weight vectors. Empirically performs better than L1. The fit for L2 will be more precise than L1. Works with all forms of training. Smoother: fewer large residual values along with fewer very small residuals as well. L2-norm has analytical solution - allows the L2-norm solutions to be calculated computationally efficiently.
* **Disadvantages**: Sensitive to outliers, since L2 wants all errors to be tiny and heavily penalizes anyone who doesn't obey. Computation heavy compared to the L1 norm. Doesn’t give you implicit feature selection.
* **Use Cases**: Use ridge if all the features are correlated with the label, as the coefficients are never zero in ridge. 
* **Relationship to Dropout**: Dropout is nothing more than an adaptive form of L2 regularization and that both methods have similar effects.


## Elastic Net

* Linear combination of L1 and L2

## Lambda Value

* Lambda is a regularization hyperparameter
* Reasonable values of lambda range between 0 and 0.1
* L2 weight regularization with very small regularization hyperparameters such as (e.g. 0.0005 or 5 x 10^−4) may be a good starting point

# RNN Model

## Import & Prepare Data

In [0]:
import tensorflow as tf
import datetime, os

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Choose Regularization

The Dense layer takes three regularizers, which all default to None. 
* **kernel_regularizer**: Regularizer function applied to the kernel weights matrix.
* **bias_regularizer**: Regularizer function applied to the bias vector.
* **activity_regularizer**: Regularizer function applied to the output of the layer (its "activation")

In [0]:
kernel_regularizer=tf.keras.regularizers.l2(l=0.0005)
bias_regularizer=None
activity_regularizer=None

**Add Dropout (optional)**

In [0]:
dropout = 0.0

## Define Model & Run

In [6]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(512, activation='relu', 
                                kernel_regularizer=kernel_regularizer, 
                                bias_regularizer=bias_regularizer, 
                                activity_regularizer=activity_regularizer))
model.add(tf.keras.layers.Dropout(dropout))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=5, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 76us/sample - loss: 0.6529 - acc: 0.8223 - val_loss: 0.5510 - val_acc: 0.8379
Epoch 2/5
60000/60000 [==============================] - 4s 73us/sample - loss: 0.5026 - acc: 0.8523 - val_loss: 0.5230 - val_acc: 0.8393
Epoch 3/5
60000/60000 [==============================] - 5s 77us/sample - loss: 0.4688 - acc: 0.8608 - val_loss: 0.4816 - val_acc: 0.8568
Epoch 4/5
60000/60000 [==============================] - 4s 74us/sample - loss: 0.4443 - acc: 0.8661 - val_loss: 0.4474 - val_acc: 0.8653
Epoch 5/5
60000/60000 [==============================] - 5s 78us/sample - loss: 0.4270 - acc: 0.8706 - val_loss: 0.4696 - val_acc: 0.8566
